In [1]:
import json
import os
from dotenv import load_dotenv
from openai import OpenAI
from pprint import pprint
import pandas as pd
import re

In [2]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openai_client = OpenAI(api_key=OPENAI_API_KEY)

In [4]:
with open("../test_cases/111025_augmented_kb.json", "r") as f:
    knowledge_base = json.load(f)
    stm_data = knowledge_base["STM_data"]
    ltm_data = knowledge_base["LTM_data"]
    hcm_data = knowledge_base["HCM_data"]
    test_profile = knowledge_base["test_elderly_profile"]


sample_stm = stm_data[:5]
sample_ltm = ltm_data[:5]
sample_hcm = hcm_data[:5]

print("\n Sample LTM \n")
pprint(sample_stm)
print("\n Sample LTM \n")
pprint(sample_ltm)
print("\n Sample LTM \n")
pprint(sample_hcm)
print("\n Test Profile \n")
pprint(test_profile)


 Sample LTM 

[{'content': 'I bought fresh fish from the wet market today; the seller was '
             'very friendly and offered a small discount.',
  'document_id': 'STM_001',
  'timestamp': '2024-05-23T08:15:00Z'},
 {'content': 'I took the MRT to Orchard Road and asked a young man about the '
             'best route to get there.',
  'document_id': 'STM_002',
  'timestamp': '2024-05-23T10:30:00Z'},
 {'content': 'I joined the tai chi session at the community centre; it felt '
             'relaxing to move gently with others.',
  'document_id': 'STM_003',
  'timestamp': '2024-05-23T16:00:00Z'},
 {'content': 'Walked to the nearby park and enjoyed some sunshine while '
             'chatting with a fellow walker.',
  'document_id': 'STM_004',
  'timestamp': '2024-05-24T10:00:00Z'},
 {'content': 'Had kopi and soft-boiled eggs at the coffee shop; the kopi was '
             'just the way I like it, strong and sweet.',
  'document_id': 'STM_005',
  'timestamp': '2024-05-24T07:45:00Z'}

In [7]:
def find_related_entries(kb):
    related_groups = []
    for stm in kb["STM_data"]:
        for ltm in kb["LTM_data"]:
            if any(word.lower() in stm["content"].lower() for word in ltm["value"].split()):
                related_groups.append({"stm": stm, "ltm": ltm})
    return related_groups

related_groups = find_related_entries(knowledge_base)

In [12]:
from RAG.utils.embedder import Embedder
from sentence_transformers import SentenceTransformer, util

embedder = Embedder()

stm_embeddings = [embedder.embed(x['content']) for x in knowledge_base['STM_data']]
ltm_embeddings = [embedder.embed(x['value']) for x in knowledge_base['LTM_data']]

INFO:root:Loading embedding model: google/embeddinggemma-300m
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: google/embeddinggemma-300m
INFO:sentence_transformers.SentenceTransformer:14 prompts are loaded, with the keys: ['query', 'document', 'BitextMining', 'Clustering', 'Classification', 'InstructionRetrieval', 'MultilabelClassification', 'PairClassification', 'Reranking', 'Retrieval', 'Retrieval-query', 'Retrieval-document', 'STS', 'Summarization']
INFO:root:embedding model loaded successfully: google/embeddinggemma-300m


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
import torch
pairs = []
for i, stm_emb in enumerate(stm_embeddings):
    sims = util.cos_sim(stm_emb, ltm_embeddings)[0]
    best_idx = torch.argmax(sims).item()
    if sims[best_idx] > 0.7:
        pairs.append({"stm": knowledge_base["STM_data"][i], "ltm": knowledge_base["LTM_data"][best_idx]})

### Generate HCM Recall Test Cases

In [18]:
HCM_recall_test_case_prompt = f"""
You are an expert data generator for generating synthetic test cases for an elderly care conversational RAG System

This is the elderly person you are emulating: {json.dumps(test_profile, indent=2)}
This is their Healthcare Memory Data: {json.dumps(hcm_data, indent=2)}

Return the requested number of test cases where the queries are realistic, conversational query based on the healthcare memory data provided. The queries should be something an elderly person might naturally ask or say in a conversation with a companion about their health, medications, or medical procedures. You may choose to include one or more relevant healthcare memory entries in each query as the expected retrieval

Each test case should be in JSON format and have the following structure. Return your answer as a JSON array of test cases.
For example:
[{{
    "test_case_id": "HCM_TC_001",
    "query": "What medications am I currently taking?",
    "expected_retrieval": "hcm": [
        {{
        "document_id": "HCM_001",
        "type": "medication",
        "date": "2022-11-05",
        "description": "Underwent cataract surgery on right eye, successful recovery."
        }},
        {{
        "document_id": "HCM_002",
        "type": "medication",
        "date": "None",
        "description": "Prescribed Glucosamine supplements for joint health."
        }}
      ]
}}]
"""

response_hcm_test_cases = openai_client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": HCM_recall_test_case_prompt},
        {"role": "user", "content": "Generate 10 HCM recall test cases.  Output only the JSON output"}
    ],
    temperature=0.8,
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [20]:
generated_hcm_tc = response_hcm_test_cases.choices[0].message.content
parsed_hcm_tc = json.loads(generated_hcm_tc)

pprint(parsed_hcm_tc)

[{'expected_retrieval': {'hcm': [{'date': 'None',
                                  'description': 'Takes Amlodipine 5mg once '
                                                 'daily for blood pressure '
                                                 'management.',
                                  'document_id': 'HCM_002',
                                  'type': 'medication'}]},
  'query': 'Can you remind me what medication I take for my blood pressure?',
  'test_case_id': 'HCM_TC_001'},
 {'expected_retrieval': {'hcm': [{'date': '2022-11-05',
                                  'description': 'Underwent cataract surgery '
                                                 'on right eye, successful '
                                                 'recovery.',
                                  'document_id': 'HCM_004',
                                  'type': 'procedure'}]},
  'query': 'When was my last cataract surgery and how did it go?',
  'test_case_id': 'HCM_TC_002'},
 {'expect

### Generate LTM-Recall Test Cases

In [23]:
LTM_recall_test_case_prompt = f"""
You are an expert data generator for generating synthetic test cases for an elderly care conversational RAG System

This is the elderly person you are emulating: {json.dumps(test_profile, indent=2)}
This is their Long Term Memory Data: {json.dumps(ltm_data, indent=2)}

Return the requested number of test cases where the queries are realistic, conversational query based on the healthcare memory data provided. The queries should be something an elderly person might naturally ask or say in a conversation with a companion about their long-term memories, life events, or personal history. You may choose to include one or more relevant long-term memory entries in each query as the expected retrieval

Each test case should be in JSON format and have the following structure. Return your answer as a JSON array of test cases.
For example:
[{{
    "id": "LTM_TC_001",
    "query": "What do I usually do on weekends?",
    "expected_retrieval": {{
      "ltm": [
        {{
          "document_id": "LTM005",
          "key": "family_relationship",
          "value": "Children and grandchildren usually join birthday celebrations"
        }}
      ]
    }}
      ]
}}]
"""

response_ltm_test_cases = openai_client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": LTM_recall_test_case_prompt},
        {"role": "user", "content": "Generate 20 LTM recall test cases.  Output only the JSON output"}
    ],
    temperature=0.8,
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [24]:
generated_ltm_tc = response_ltm_test_cases.choices[0].message.content
parsed_ltm_tc = json.loads(generated_ltm_tc)

pprint(parsed_ltm_tc)

[{'expected_retrieval': {'ltm': [{'document_id': 'LTM_001',
                                  'key': 'Husband',
                                  'value': 'Rajinder Singh'}]},
  'id': 'LTM_TC_001',
  'query': 'Who is my husband and what can you tell me about him?'},
 {'expected_retrieval': {'ltm': [{'document_id': 'LTM_002',
                                  'key': 'Eldest son',
                                  'value': 'Harjit Singh'},
                                 {'document_id': 'LTM_003',
                                  'key': 'Youngest daughter',
                                  'value': 'Simran Kaur'}]},
  'id': 'LTM_TC_002',
  'query': 'Can you remind me about my children?'},
 {'expected_retrieval': {'ltm': [{'document_id': 'LTM_005',
                                  'key': 'Morning routine',
                                  'value': 'Enjoys morning walks at Dhoby '
                                           'Ghaut Park with neighbours at 7 '
                           

In [25]:
print(len(stm_data))

300


In [32]:
all_stm_test_cases = []
for i in range(0,len(stm_data), 50):
    stm_data_chunk = stm_data[i:i+50]
    print(f"Generating test cases for STM entries {i} to {i+50}")
    STM_recall_test_case_prompt = f"""
    You are an expert data generator for generating synthetic test cases for an elderly care conversational RAG System

    This is the elderly person you are emulating: {json.dumps(test_profile, indent=2)}
    This is their Short Term Memory Data: {json.dumps(stm_data_chunk, indent=2)}

    Return the requested number of test cases where the queries are realistic, conversational query based on the healthcare memory data provided. The queries should be something an elderly person might naturally ask or say in a conversation with a companion about their short-term memories, life events, or personal history. You may to include one or more relevant short-term memory entries in each query as the expected retrieval

    Each test case should be in JSON format and have the following structure. Return your answer as a JSON array of test cases.
    For example:
    [{{
    "id": "STM_3",
    "query": "When was the health talk again?",
    "expected_retrieval": {{
      "stm": [
        {{
            "document_id": "STM_052",
            "timestamp": "2025-10-02T03:56:53Z",
            "content": "I heard there will be a health talk at the community centre next week."
        }}
      ]
    }}
    }}
    ]
    """
    response_stm_test_cases = openai_client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": STM_recall_test_case_prompt},
            {"role": "user", "content": "Generate 20 STM recall test cases.  Output only the JSON output"}
        ],
        temperature=0.8,
    )
    generated_stm_tc = response_stm_test_cases.choices[0].message.content
    parsed_stm_tc = json.loads(generated_stm_tc)
    all_stm_test_cases.extend(parsed_stm_tc)
    print(f"Generated {len(parsed_stm_tc)} test cases")
    print(f"Total test cases so far: {len(all_stm_test_cases)}")




Generating test cases for STM entries 0 to 50


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Generated 20 test cases
Total test cases so far: 20
Generating test cases for STM entries 50 to 100


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Generated 20 test cases
Total test cases so far: 40
Generating test cases for STM entries 100 to 150


...

Generated 20 test cases
Total test cases so far: 80
Generating test cases for STM entries 200 to 250


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Generated 20 test cases
Total test cases so far: 100
Generating test cases for STM entries 250 to 300


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Generated 20 test cases
Total test cases so far: 120


In [35]:
for idx, tc in enumerate(all_stm_test_cases, 1):
    tc['id'] = f"STM_TC_{idx:03d}"

In [36]:
pprint(all_stm_test_cases)
print(len(all_stm_test_cases))

[{'expected_retrieval': {'stm': [{'content': 'I remembered to take my heart '
                                             'medicine after breakfast today; '
                                             'feeling better each day.',
                                  'document_id': 'STM_007',
                                  'timestamp': '2024-05-25T09:00:00Z'}]},
  'id': 'STM_TC_001',
  'query': 'Did I remember to take my heart medicine after breakfast the other '
           'day?'},
 {'expected_retrieval': {'stm': [{'content': 'Cooked dal with extra garlic '
                                             'today; my family said it tasted '
                                             'comforting and warm.',
                                  'document_id': 'STM_010',
                                  'timestamp': '2024-05-25T18:00:00Z'}]},
  'id': 'STM_TC_002',
  'query': 'What did I cook with extra garlic that my family liked?'},
 {'expected_retrieval': {'stm': [{'content': 'Asked the bus

### Consolidate and save all test cases

In [37]:
all_augmented_test_cases = []
all_augmented_test_cases.extend(parsed_hcm_tc)
all_augmented_test_cases.extend(parsed_ltm_tc)
all_augmented_test_cases.extend(all_stm_test_cases)

print(len(all_augmented_test_cases))

150


In [38]:
with open("../test_cases/111025_augmented_test_cases.json", "w") as f:
    json.dump(all_augmented_test_cases, f, indent=4)